In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
melb_data = pd.read_csv("../input/melbourne-housing-snapshot/melb_data.csv")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [ ]:
X_train.head()

In [ ]:
# Benefits of Pipeline
# 1. Cleaner Code
# 2. Fewer Bugs
# 3. Easier to Productionize
# 4. More Options for Model Validation

In [ ]:
from sklearn.compose import ColumnTransformer # Preprocessing for Categorical Data
from sklearn.pipeline import Pipeline # Bundle preprocessing for numerical and categorical data
from sklearn.impute import SimpleImputer # Imputation
from sklearn.preprocessing import OneHotEncoder # Categorical Variables

# Prerpocessing for numercial data
numerical_transformer = SimpleImputer(strategy='constant') # default (strategy='constant', fill_value = 0)

# Proprocessing for categorical data
categorical_transformer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent')),('onehot',OneHotEncoder(handle_unknown='ignore'))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(transformers = [('num',numerical_transformer, numerical_cols),('cat',categorical_transformer, categorical_cols)])

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [ ]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps = [('preprocessor',preprocessor),('model',model)])

# Preprocessing of training data, fir model
my_pipeline.fit(X_train, y_train)

# Prerpocssing of validation data, get prediction
preds = my_pipeline.predict(X_valid)

# Evaluation the model
score = mean_absolute_error(y_valid, preds)
print("MAE:",score)